In [10]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

In [ ]:
when N N -> N N
when A N -> N N

when N N -> N N
when A N -> A N


In [11]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,10,20,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [12]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

In [13]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()

corpus_list<-c("google")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("Tagged")
compounds_list<-c("all","an","nn")
ppmi_setting_list<-c("RAW")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [14]:
for (c in corpus_list){
  for (t in tagged_list) {
    for (f in feature_setting_list) {
      for (p in ppmi_setting_list){
        for (a in comp_setting_list){
          for (i in time_off) {
            
            for (j in cut_off) {
              
              
              print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
              
              na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
              nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
              
              for (r in ratings_list){
                na_df<-na_df %>% filter(source==r)
                nona_df<-nona_df %>% filter(source==r)
                
                for (k in compounds_list) {
                  if (k=="an") { 
                    group1_na<-na_df %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_na<-na_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    na_df<-rbind(group1_na,group2_na)
                    
                    group1_nona<-nona_df %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_nona<-nona_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    nona_df<-rbind(group1_nona,group2_nona)
                    
                  }
                  else if (k=="nn") {
                    group1_na<-na_df %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_na<-na_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    na_df<-rbind(group1_na,group2_na)
                    
                    group1_nona<-nona_df %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_nona<-nona_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    nona_df<-rbind(group1_nona,group2_nona)
                    
                  }
                  
                  trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  trainY_na<-na_df %>% select(compositionality)
                  trainY_na<-trainY_na$compositionality
                  
                  
                  trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  trainY_nona<-nona_df %>% select(compositionality)
                  trainY_nona<-trainY_nona$compositionality
                  
                  if (dim(trainX_nona)[1]<10) {
                    break
                  }
                  for (s in seed_list)  {
                    set.seed(s)
                    seeds <- vector(mode = "list", length = 14)
                    for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                    #for the last model
                    seeds[[14]]<-sample.int(1000, 1)
                    
                    ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                    )
                    
                    elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                          trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                          preProc = c("nzv","center", "scale")
                    )
                    
                    lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                    )
                    
                    
                    ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                        preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    
                    ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                    )
                    
                    elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                   trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                   preProc = c("nzv","center", "scale")
                    )
                    
                    lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                    )
                    
                    
                    ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                 preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    
                    perf_elastic_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                    perf_ridge_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                    perf_lasso_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                    
                    
                    perf_elastic_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                    perf_ridge_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                    perf_lasso_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                    
                    varimp_elastic_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_na)$importance))
                    varimp_ridge_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_na)$importance))
                    varimp_lasso_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_na)$importance))
                    
                    varimp_elastic_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_nona)$importance))
                    varimp_ridge_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_nona)$importance))
                    varimp_lasso_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_nona)$importance))    
                    
                    
                    
                    list_of_rsqr[[m]]<-perf_elastic_na
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_elastic_nona
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_ridge_na
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_ridge_nona
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_lasso_na
                    m<-m+1 
                    
                    list_of_rsqr[[m]]<-perf_lasso_nona
                    m<-m+1 
                    
                    if (i==10000) {
                      list_of_vi_10000[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_lasso_nona
                      v<-v+1
                      
                    }
                    else if (i==10) {
                      list_of_vi_10[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    
                    else if (i==20) {
                      list_of_vi_20[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    else if (i==50) {
                      list_of_vi_50[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    else if (i==100) {
                      list_of_vi_100[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                  }
                }
                
              }
            }
          }
        }
      }
    }
  }
}

[1] "google RAW Aware withSetting Tagged 10000 0"
[1] "google RAW Aware withSetting Tagged 10000 10"
[1] "google RAW Aware withSetting Tagged 10000 50"
[1] "google RAW Aware withSetting Tagged 10000 100"
[1] "google RAW Aware withSetting Tagged 10000 500"
[1] "google RAW Aware withSetting Tagged 10000 1000"
[1] "google RAW Aware withSetting Tagged 10 0"
[1] "google RAW Aware withSetting Tagged 10 10"
[1] "google RAW Aware withSetting Tagged 10 50"
[1] "google RAW Aware withSetting Tagged 10 100"
[1] "google RAW Aware withSetting Tagged 10 500"
[1] "google RAW Aware withSetting Tagged 10 1000"


In [16]:
head(rsquared_df)

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
1,withSetting,r,all,Aware,reddy,google,elastic,364,1001,10000,0,RAW,glmnet,0.3336010,0.5447552
2,withSetting,python,all,Aware,reddy,google,elastic,364,1001,10000,0,RAW,glmnet,0.3271287,0.5337331
3,withSetting,r,all,Aware,reddy,google,rigde,364,1001,10000,0,RAW,glmnet,0.3274767,0.5361013
4,withSetting,python,all,Aware,reddy,google,rigde,364,1001,10000,0,RAW,glmnet,0.3231517,0.5313964
5,withSetting,r,all,Aware,reddy,google,lasso,364,1001,10000,0,RAW,glmnet,0.3299949,0.5407521
6,withSetting,python,all,Aware,reddy,google,lasso,364,1001,10000,0,RAW,glmnet,0.3238064,0.5310675


In [18]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
statistics<-rsquared_df %>% group_by(tag,impute,setting,dataset,ml_algo,timespan,cutoff) %>% 
summarise(n=mean(n),meanRsquared=mean(TrainRsquared),sdRsquared=sd(TrainRsquared),meanCorr=mean(TrainSpearman),sdCorr=sd(TrainSpearman))

statistics %>% arrange(desc(meanCorr)) %>% head()

`summarise()` has grouped output by 'tag', 'impute', 'setting', 'dataset', 'ml_algo', 'timespan'. You can override using the `.groups` argument.


tag,impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
an,python,Aware,reddy,elastic,10,10,91,0.5475358,0.03419424,0.7284653,0.04026657
nn,python,Aware,reddy,elastic,10,10,91,0.5475358,0.03419424,0.7284653,0.04026657
an,python,Aware,reddy,lasso,10,10,91,0.5458346,0.03485322,0.7190402,0.04262364
nn,python,Aware,reddy,lasso,10,10,91,0.5458346,0.03485322,0.7190402,0.04262364
an,r,Aware,reddy,elastic,10,10,91,0.5224436,0.03769707,0.7060567,0.03835683
nn,r,Aware,reddy,elastic,10,10,91,0.5224436,0.03769707,0.7060567,0.03835683


In [8]:
varimp_df<-bind_rows(list_of_vi_10000)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0

In [9]:
varimp_df

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,⋯,ppmi.0,sim_bw_constituents.0,sim_cpf_0.0,sim_cpf_100.0,sim_cpf_25.0,sim_cpf_50.0,sim_cpf_75.0,sim_cpf_beta.0,sim_with_head.0,sim_with_modifier.0
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Overall...1,woSetting,r,all,Aware,reddy,google,elastic,364,1001,10000,⋯,66.30599,8.981281,54.01845,13.9363584,70.06499,74.04475,1.457394,49.12701,32.038018,28.79514
Overall...2,woSetting,python,all,Aware,reddy,google,elastic,364,1001,10000,⋯,76.12790,5.163326,24.89847,7.9640406,31.14658,34.18288,7.212006,19.14151,58.924396,54.35592
Overall...3,woSetting,r,all,Aware,reddy,google,rigde,364,1001,10000,⋯,84.04803,17.432234,62.89377,0.5346305,40.16508,45.04636,27.250859,69.50448,13.975093,27.40277
Overall...4,woSetting,python,all,Aware,reddy,google,rigde,364,1001,10000,⋯,100.00000,21.743339,67.06178,1.2169384,14.47413,14.71626,25.240317,48.05124,6.692141,26.48826
Overall...5,woSetting,r,all,Aware,reddy,google,lasso,364,1001,10000,⋯,12.15197,3.845368,20.86623,8.8559099,34.97012,43.09219,14.366769,16.02533,61.476062,55.43830
Overall...6,woSetting,python,all,Aware,reddy,google,lasso,364,1001,10000,⋯,48.82480,4.021275,16.64232,4.4691453,16.42702,15.40532,0.000000,13.44502,59.872423,55.56154
Overall...7,woSetting,r,all,Aware,reddy,google,elastic,364,1111,10000,⋯,65.22837,10.952861,52.89449,7.1974893,46.37549,49.18274,14.465487,56.32743,5.574646,11.84977
Overall...8,woSetting,python,all,Aware,reddy,google,elastic,364,1111,10000,⋯,65.24542,4.826802,21.11973,5.4438119,21.85237,20.45727,0.000000,16.84955,59.269200,55.11991
Overall...9,woSetting,r,all,Aware,reddy,google,rigde,364,1111,10000,⋯,84.04803,17.432234,62.89377,0.5346305,40.16508,45.04636,27.250859,69.50448,13.975093,27.40277


In [7]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)

In [9]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
644,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1771,20,50,PPMI,glmnet,0.5279124,0.7313636
476,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,100,PPMI,glmnet,0.5225827,0.7200233
356,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,10,PPMI,glmnet,0.5231564,0.7131652
175,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,50,PPMI,glmnet,0.5585807,0.7128805
177,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,50,PPMI,glmnet,0.5399065,0.7128805
422,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1001,10,100,PPMI,glmnet,0.5311879,0.7118798
235,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,100,PPMI,glmnet,0.5597754,0.7109973
55,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,0,PPMI,glmnet,0.5582943,0.7095471
57,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,0,PPMI,glmnet,0.5399696,0.7095471


In [10]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
440,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,100,PPMI,glmnet,0.6028381,0.6806244
620,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,50,PPMI,glmnet,0.5885712,0.6776766
320,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,10,PPMI,glmnet,0.5854357,0.6588983
380,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,50,PPMI,glmnet,0.5837606,0.6813442
324,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,10,PPMI,glmnet,0.5819766,0.6440176
444,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,100,PPMI,glmnet,0.5814872,0.6782434
624,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,20,50,PPMI,glmnet,0.5751432,0.6693253
384,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,50,PPMI,glmnet,0.5744349,0.6491037
680,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,100,PPMI,glmnet,0.5685864,0.6998218


In [19]:
write.csv(rsquared_df,"rsquared_Tagged_RAW_Agnostic.csv",row.names = FALSE)

In [20]:
write.csv(varimp_df,"varimp_Tagged_RAW_Agnostic.csv",row.names = FALSE)